In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

num_qubits = 7
dataset_filename = "dataset/dataset_tesi/NN1_Dataset(<=10Cx)_balanced1.csv"
df = pd.read_csv(dataset_filename)

links = [set([0,1]), set([1,2]), set([1,3]), set([3,5]), set([4,5]), set([5,6])]
def generate_columns(header, links, in_links=False):
    if in_links:
        return [header+str(i)+str(j) for i in range(num_qubits) for j in range(num_qubits) if set([i,j]) in links]
    else:
        return [header+str(i)+str(j) for i in range(num_qubits) for j in range(num_qubits) if set([i,j]) not in links and i!=j]

useless_columns = ['Unnamed: 0', 'last_update_date', 'N_qubtis', 'N_measure', 'N_cx', 'backend_name']
useless_columns += generate_columns("cx_", links)
useless_columns += generate_columns("edge_length_", links)
useless_columns += generate_columns("edge_error_", links)
useless_columns += ["measure_"+str(i) for i in range(num_qubits)]
# Note that cx/edge_error/edge_length_xy is not neccessarily the same as cx/edge_length/edge_error_yx
df.drop(columns=useless_columns, inplace=True)
df.drop_duplicates(inplace=True)

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train, test = train_test_split(df, test_size=0.2)
df_train_x, df_train_y= train.iloc[:, :-num_qubits], train.iloc[:, -num_qubits:]
df_test_x, df_test_y= test.iloc[:, :-num_qubits], test.iloc[:, -num_qubits:]

train_x = scaler.fit_transform(df_train_x)
test_x = scaler.fit_transform(df_test_x)


# for every row in y, convert to 1 hot-encoding and flatten
train_y = []
for _, row in df_train_y.iterrows():
    train_y.append(pd.get_dummies(row).values.flatten())
train_y = np.array(train_y)

test_y = []
for _, row in df_test_y.iterrows():
    test_y.append(pd.get_dummies(row).values.flatten())
test_y = np.array(test_y)

(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

((7026, 57), (7026, 49), (1757, 57), (1757, 49))

# Simple MLP

In [3]:
import torch
from torch.nn import Linear
import torch.nn.functional as F


class MLP(torch.nn.Module):
    def __init__(self, num_features, num_classes, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.lin1 = Linear(num_features, hidden_channels)
        self.lin2 = Linear(hidden_channels, hidden_channels)
        self.lin3 = Linear(hidden_channels, hidden_channels)
        self.lin4 = Linear(hidden_channels, num_classes)

    def forward(self, x):
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        x = self.lin3(x)
        x = x.relu()
        x = self.lin4(x)
        return x

/home/jacky/anaconda3/envs/eece571f/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [49]:
model = MLP(57, 49, hidden_channels=128)
criterion = torch.nn.CrossEntropyLoss()  # Define loss criterion.
optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=5e-4)  # Define optimizer.

torch_train_x = torch.tensor(train_x, dtype=torch.float)
torch_train_y = torch.tensor(train_y, dtype=torch.float)
torch_test_x = torch.tensor(test_x, dtype=torch.float)
torch_test_y = torch.tensor(test_y, dtype=torch.float)

In [50]:
def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(torch_train_x)  # Perform a single forward pass.
      loss = criterion(out, torch_train_y)  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(x, y):
      model.eval()
      pred = model(x)

      test_correct = 0
      for i, j in zip(pred, y):
          pred_i = np.argmax(i.detach().numpy().reshape(7,7), axis=1)
          label_j = np.argmax(j.detach().numpy().reshape(7,7), axis=1)
          test_correct += np.array_equal(pred_i, label_j)
      test_acc = int(test_correct) / int(y.shape[0])  # Derive ratio of correct predictions.
      return test_acc


In [51]:
for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 27.0800
Epoch: 002, Loss: 21.9036
Epoch: 003, Loss: 21.3844
Epoch: 004, Loss: 20.5604
Epoch: 005, Loss: 19.8649
Epoch: 006, Loss: 19.5504
Epoch: 007, Loss: 19.3385
Epoch: 008, Loss: 18.7612
Epoch: 009, Loss: 18.3172
Epoch: 010, Loss: 17.9141
Epoch: 011, Loss: 17.4990
Epoch: 012, Loss: 17.2550
Epoch: 013, Loss: 17.1155
Epoch: 014, Loss: 17.0426
Epoch: 015, Loss: 17.0478
Epoch: 016, Loss: 17.0295
Epoch: 017, Loss: 16.9399
Epoch: 018, Loss: 16.8995
Epoch: 019, Loss: 16.8222
Epoch: 020, Loss: 16.7528
Epoch: 021, Loss: 16.6812
Epoch: 022, Loss: 16.6238
Epoch: 023, Loss: 16.5539
Epoch: 024, Loss: 16.4822
Epoch: 025, Loss: 16.4323
Epoch: 026, Loss: 16.3780
Epoch: 027, Loss: 16.3590
Epoch: 028, Loss: 16.3174
Epoch: 029, Loss: 16.2600
Epoch: 030, Loss: 16.2344
Epoch: 031, Loss: 16.1893
Epoch: 032, Loss: 16.1512
Epoch: 033, Loss: 16.1436
Epoch: 034, Loss: 16.1210
Epoch: 035, Loss: 16.0966
Epoch: 036, Loss: 16.0782
Epoch: 037, Loss: 16.0537
Epoch: 038, Loss: 16.0302
Epoch: 039, 

In [54]:
training_acc = test(torch_train_x, torch_train_y)
print(f'Training Accuracy: {training_acc:.4f}')

testing_acc = test(torch_test_x, torch_test_y)
print(f'Training Accuracy: {testing_acc:.4f}')

Training Accuracy: 0.9069
Training Accuracy: 0.8645


# Simple GNN

In [ ]:
node_prefix = ["T1", "T2", "readout_error"]
node_features_x = []
for k in range(df_train_x.shape[0]):
    node_features_j = []
    for j in range(num_qubits):
        row_features = df_train_x.iloc[k][[i + "_" + str(j) for i in node_prefix]].values.flatten()
        node_features_j.append(row_features)
    node_features_j = scaler.fit_transform(node_features_j)
    node_features_x.append(node_features_j)
node_features_x = np.array(node_features_x)

edge_prefix = ["cx_", "edge_length_", "edge_error_"]
edge_index = [[],[]]
edge_features_x = []
for k in range(df_train_x.shape[0]):
    edge_features_j = []
    for i in range(num_qubits):
        for j in range(num_qubits):
            if set([i,j]) in links:
                row_features = df_train_x.iloc[k][[prefix + str(i) + str(j) for prefix in edge_prefix]].values.flatten()
                edge_features_j.append(row_features)
                if k == 0: # only need to do this once
                    edge_index[0].append(i)
                    edge_index[1].append(j)
    edge_features_j = scaler.fit_transform(edge_features_j)
    edge_features_x.append(edge_features_j)
edge_features_x = np.array(edge_features_x)
edge_index = np.array(edge_index)

node_labels = df_train_y.to_numpy()
print(node_features_x.shape, edge_index.shape, edge_features_x.shape, node_labels.shape)

In [ ]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(3, 2)
        self.conv2 = GCNConv(2, 2)
        self.conv3 = GCNConv(2, 1)
        self.classifier = Linear(1, 7)

    def forward(self, x, edge_index, edge_attr):
        h = self.conv1(x, edge_index, edge_attr)
        h = h.tanh()
        h = self.conv2(h, edge_index, edge_attr)
        h = h.tanh()
        h = self.conv3(h, edge_index, edge_attr)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

In [ ]:
torch_node_features_x = torch.tensor(node_features_x, dtype=torch.float)
torch_edge_index = torch.tensor(edge_index, dtype=torch.long)
torch_edge_features_x = torch.tensor(edge_features_x, dtype=torch.float)
torch_node_labels = torch.tensor(node_labels, dtype=torch.long)

In [46]:
model = GCN()
criterion = torch.nn.CrossEntropyLoss()  #Initialize the CrossEntropyLoss function.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Initialize the Adam optimizer.

def train(node_features, edge_index, edge_attr, node_labels):

    # for i in range(node_features.shape[0]):
    optimizer.zero_grad()  # Clear gradients.
    out, h = model(node_features, edge_index, edge_attr)  # Perform a single forward pass.
    loss = criterion(out, node_labels)  # Compute the loss solely based on the training nodes.
    loss.backward()  # Derive gradients.
    optimizer.step()  # Update parameters based on gradients.

    return loss, h


for epoch in range(401):
    #torch_edge_weight = torch.tensor(np.sum(edge_features_x, axis=2), dtype=torch.float)
    torch_edge_weight = torch.tensor(np.random.randn(12,1), dtype=torch.float)
    loss, h = train(torch_node_features_x[0], torch_edge_index, torch_edge_weight, torch_node_labels[0])
    print(f'Epoch: {epoch}, Loss: {loss}')

Epoch: 0, Loss: nan
Epoch: 1, Loss: nan
Epoch: 2, Loss: nan
Epoch: 3, Loss: nan
Epoch: 4, Loss: nan
Epoch: 5, Loss: nan
Epoch: 6, Loss: nan
Epoch: 7, Loss: nan
Epoch: 8, Loss: nan
Epoch: 9, Loss: nan
Epoch: 10, Loss: nan
Epoch: 11, Loss: nan
Epoch: 12, Loss: nan
Epoch: 13, Loss: nan
Epoch: 14, Loss: nan
Epoch: 15, Loss: nan
Epoch: 16, Loss: nan
Epoch: 17, Loss: nan
Epoch: 18, Loss: nan
Epoch: 19, Loss: nan
Epoch: 20, Loss: nan
Epoch: 21, Loss: nan
Epoch: 22, Loss: nan
Epoch: 23, Loss: nan
Epoch: 24, Loss: nan
Epoch: 25, Loss: nan
Epoch: 26, Loss: nan
Epoch: 27, Loss: nan
Epoch: 28, Loss: nan
Epoch: 29, Loss: nan
Epoch: 30, Loss: nan
Epoch: 31, Loss: nan
Epoch: 32, Loss: nan
Epoch: 33, Loss: nan
Epoch: 34, Loss: nan
Epoch: 35, Loss: nan
Epoch: 36, Loss: nan
Epoch: 37, Loss: nan
Epoch: 38, Loss: nan
Epoch: 39, Loss: nan
Epoch: 40, Loss: nan
Epoch: 41, Loss: nan
Epoch: 42, Loss: nan
Epoch: 43, Loss: nan
Epoch: 44, Loss: nan
Epoch: 45, Loss: nan
Epoch: 46, Loss: nan
Epoch: 47, Loss: nan
Ep